In [1]:
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

     |████████████████████████████████| 3.1 MB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 88.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 19.2 MB 23.3 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=149984bd26f95005524899f30f3df42a7eff6e03b32d07c2911817f7e2f19bcf
  Stored in directory: /root/.cache/pip/wheels/ca/38/d8/dfdfe73bec1d12026b30cb7ce8da650f3f0ea2cf155ea018ae
Successfully built jieba
     |████████████████████████████████| 108.2 MB 90.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 83.7 MB/s eta 0:00:01


In [1]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import FusionEmedding,GlyphEmbedding,PinyinEmbedding
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
with open('AnchiBERT/vocab.txt','r') as f:
    ix2BertChar = defaultdict(lambda: '[UNK]') 
    ix2BertChar.update({ i: _ for (i,_) in enumerate(f.readlines())})
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
glyph = GlyphEmbedding('data/glyph_weight.npy')
pinyinV = PinyinEmbedding(30,'data/pinyin_map.json')
# Example
display(glyph(torch.tensor([0,1,2])))
display(pinyinV(torch.tensor([0,1])))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0421, -0.0327,  0.0096,  ...,  0.0711,  0.0417, -0.0224],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.2590,  0.2719,  0.1614,  0.0099,  0.1555, -0.2728,  0.2700, -0.1006,
         -0.0885,  0.0607,  0.2121,  0.1352,  0.1809, -0.0842,  0.2739, -0.2115,
         -0.2756, -0.1202,  0.0751, -0.2282, -0.2437, -0.2295,  0.2278, -0.2172,
          0.0318, -0.2727,  0.0463, -0.2532, -0.1979,  0.1673]],
       grad_fn=<EmbeddingBackward>)

In [6]:
# Example
_ = tokenizer('[UNK]辛[UNK]业怠π',return_tensors='pt')
display(_)
Anchibert(**_).last_hidden_state.shape

{'input_ids': tensor([[ 101,  100, 6789,  100,  689, 2591,  223,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

torch.Size([1, 8, 768])

In [2]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [4]:
trainSet = FusionDataset(tr_in[:110],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[:110],tr_pos_in[:110],tr_pos_out[:110])

110it [00:00, 2097.24it/s]
110it [00:00, 2196.09it/s]


In [5]:
trainLoader = DataLoader(trainSet,batch_size=64,shuffle=True)
#     validLoader = DataLoader(validSet,batch_size=batch_size,shuffle=False)

In [26]:
config = {
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6,
    'output_dim':len(ix2BertChar)
}


In [41]:
class Pinyin_Glyph_Pos_Anchi_Transformer(nn.Module):
    def __init__(self,config):
        super().__init__()
        
        self.embedding = FusionEmedding(config)
        decoder_layer = nn.TransformerDecoderLayer(d_model=config['hidden_size'], nhead=config['nhead'])
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config['num_layers'])
        
        self.Linear = nn.Linear(config['hidden_size'],config['output_dim'])
    def forward(self,Xword_embeddings,Xsents_pinyin_ids, \
                Xsents_token_type_ids,Xsents_pos_ids,\
                Yword_embeddings,Ysents_pinyin_ids, \
                Ysents_token_type_ids,Ysents_pos_ids):
        
        memory = self.embedding(Xword_embeddings,Xsents_pinyin_ids, \
                                Xsents_token_type_ids,Xsents_pos_ids).permute([1,0,2])
        tgt = self.embedding(Yword_embeddings,Ysents_pinyin_ids, \
                             Ysents_token_type_ids,Ysents_pos_ids).permute([1,0,2])
        outputs = self.transformer_decoder(tgt, memory)
        outputs = self.Linear(out)
        
        return F.log_softmax(outputs, dim=2)

In [42]:
model = Pinyin_Glyph_Pos_Anchi_Transformer(config)
model.to(device)

Pinyin_Glyph_Pos_Anchi_Transformer(
  (embedding): FusionEmedding(
    (position_embeddings): Embedding(50, 768)
    (glyph_embeddings): GlyphEmbedding(
      (embedding): Embedding(9110, 576)
    )
    (pinyin_embeddings): PinyinEmbedding(
      (embedding): Embedding(1297, 30, padding_idx=0)
    )
    (pos_tag_embeddings): Embedding(30, 10, padding_idx=0)
    (fc): Linear(in_features=1384, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=768, out_features=768, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
 

In [43]:
Anchibert.to(device)
for Xsents_input_ids,Xsents_token_type_ids,\
    Xsents_attention_mask,Xsents_pinyin_ids,\
    Xsents_glyph_ids,Xsents_pos_ids,\
    Ysents_input_ids,Ysents_token_type_ids,\
    Ysents_attention_mask,Ysents_pinyin_ids,\
    Ysents_glyph_ids,Ysents_pos_ids in trainLoader:
    
    Xsents_input_ids = Xsents_input_ids.to(device)
    Xsents_token_type_ids = Xsents_token_type_ids.to(device)
    Xsents_attention_mask = Xsents_attention_mask.to(device)
    Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
    Xsents_glyph_ids = Xsents_glyph_ids.to(device)
    Xsents_pos_ids = Xsents_pos_ids.to(device)
    Ysents_input_ids = Ysents_input_ids.to(device)
    Ysents_token_type_ids = Ysents_token_type_ids.to(device)
    Ysents_attention_mask = Ysents_attention_mask.to(device)
    Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
    Ysents_glyph_ids = Ysents_glyph_ids.to(device)
    Ysents_pos_ids = Ysents_pos_ids.to(device)
    
    Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                )['last_hidden_state'].detach()


    Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                 Ysents_token_type_ids, \
                                 Ysents_attention_mask  \
                                )['last_hidden_state'].detach()
    
    outputs = model(Xword_embeddings,Xsents_pinyin_ids, \
                    Xsents_token_type_ids,Xsents_pos_ids,\
                    Yword_embeddings,Ysents_pinyin_ids, \
                    Ysents_token_type_ids,Ysents_pos_ids)
    break

In [44]:
outputs.shape

torch.Size([28, 64, 21128])

In [10]:
model = FusionEmedding(config)
model.to(device)
Anchibert.to(device)
for Xsents_input_ids,Xsents_token_type_ids,\
    Xsents_attention_mask,Xsents_pinyin_ids,\
    Xsents_glyph_ids,Xsents_pos_ids,\
    Ysents_input_ids,Ysents_token_type_ids,\
    Ysents_attention_mask,Ysents_pinyin_ids,\
    Ysents_glyph_ids,Ysents_pos_ids in trainLoader:
    
    Xsents_input_ids = Xsents_input_ids.to(device)
    Xsents_token_type_ids = Xsents_token_type_ids.to(device)
    Xsents_attention_mask = Xsents_attention_mask.to(device)
    Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
    Xsents_glyph_ids = Xsents_glyph_ids.to(device)
    Xsents_pos_ids = Xsents_pos_ids.to(device)
    Ysents_input_ids = Ysents_input_ids.to(device)
    Ysents_token_type_ids = Ysents_token_type_ids.to(device)
    Ysents_attention_mask = Ysents_attention_mask.to(device)
    Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
    Ysents_glyph_ids = Ysents_glyph_ids.to(device)
    Ysents_pos_ids = Ysents_pos_ids.to(device)
    
    Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                )['last_hidden_state'].detach()
    outputembeddingsX = model(Xword_embeddings,Xsents_pinyin_ids, \
                             Xsents_token_type_ids,Xsents_pos_ids)

    Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                 Ysents_token_type_ids, \
                                 Ysents_attention_mask  \
                                )['last_hidden_state'].detach()
    
    outputembeddingsY = model(Yword_embeddings,Ysents_pinyin_ids, \
                             Ysents_token_type_ids,Ysents_pos_ids)
    break

In [11]:
outputembeddingsX.shape,outputembeddingsY.shape

(torch.Size([64, 29, 768]), torch.Size([64, 28, 768]))